# Analysis of the Public Threat model

> Initial results for my Public Threat model

In [1]:
#| default_exp analysis_public_threat

In [ ]:
#| hide
#| export
from gh_pages_example.conditions import *
from gh_pages_example.data_utils import *
from gh_pages_example.methods import *
from gh_pages_example.models import *
from gh_pages_example.model_utils import *
from gh_pages_example.payoffs import *
from gh_pages_example.plot_utils import *
from gh_pages_example.types import *
from gh_pages_example.utils import *

import copy
import typing
import warnings

import chaospy
import fastcore.test
import ipywidgets
from ipywidgets import interact
import matplotlib as mpl
import matplotlib.pyplot as plt
from nbdev.showdoc import *
import nptyping
import numpy as np
import pandas
import scipy
import seaborn as sns

In [ ]:
Z = {"S2": 50, "S1": 50}
β = 1
sector_strategies = {"S2": [4, 5],
                     "S1": [1, 2, 3], }
allowed_sectors = {"P3": ["S2"],
                   "P2": ["S1"],
                   "P1": ["S1"], }
models = {"dispatch-type": 'multiple-populations',
          "payoffs_key": "public_threat_level_v1",
          "β": β,
          "Z": Z,
          "allowed_sectors": allowed_sectors,
          "sector_strategies": sector_strategies,
          "compute_success_rule": "functional",
          "payoffs_function": evaluate_payoffs,
          }
models['pfo'] = 0 # We need this parameter to calculate some analytical boundaries.

In [ ]:
models_baseline = {**models,
                   **build_reg_market(pfo_h=0.8,
                                      g=1.4,
                                      incentive_mix=1,
                                      phi_h=1,
                                      phi2_h=1,
                                      β = 0.02 # np.arange(0.01, 0.1, 0.01)
                                      )
                   }
models_baseline['β'] = {"S1": models_baseline['β'],
                        "S2": models_baseline['β']*10}
# Optionally, we can choose to scale the regulator's impact by the speed advantage.
models_baseline['phi_h'] = models_baseline['phi_h']  / models_baseline['s']
models_baseline['phi2_h'] = models_baseline['phi2_h'] / models_baseline['s']
results_baseline = thread_macro({**models_baseline,
                                 "sd-method": "quantecon",
                                 },
                                risk_dominance,
                                payoffs_sr_pfo_extension,
                                threshold_risk_dominant_safety_dsair,
                                threshold_society_prefers_safety_dsair,
                                create_profiles,
                                apply_profile_filters,
                                build_payoffs,
                                build_transition_matrix,
                                find_ergodic_distribution,
                                calculate_sd_helper,
                                )
df_baseline = thread_macro(results_baseline,
                           results_to_dataframe_egt,
                           process_dsair_data)